In [3]:
import pandas as pd

In [23]:
import os
import re
import json
import pandas as pd

# 定义一个函数来提取label值
def extract_label_from_json(content):
    match = re.search(r'\{.*?\}', content, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            json_dict = json.loads(json_str)
            return json_dict.get("label", "Key not found")
        except json.JSONDecodeError:
            return "JSON decode error"
    return "No JSON content"

# 定义一个函数来比较两个label值
def compare_labels(label1, label2):
    labels1 = label1.strip().replace(' ', '').split('、')
    labels2 = label2.strip().replace(' ', '').split('、')
    
    if len(labels1) == 1 and len(labels2) == 1:
        return 1 if labels1[0] == labels2[0] else 0
    else:
        set1 = set(labels1)
        set2 = set(labels2)
        return 1 if set1 & set2 else 0

# 定义一个通用函数来处理提取、比较和统计
def process_columns(df, json_col='predict', label_col='label'):
    new_col_name = f"{json_col}_contrast"

    def process_row(row):
        json_label = extract_label_from_json(row[json_col])
        numeric_label = row[label_col] if pd.notna(row[label_col]) else "NA"
        return compare_labels(json_label, numeric_label)
    if label_col not in df.columns:
        raise KeyError(f"Column '{label_col}' not found in DataFrame.")
    
    df[new_col_name] = df.apply(process_row, axis=1)
    total_rows = len(df)
    contrast_1_count = df[new_col_name].apply(lambda x: x if isinstance(x, int) else 0).sum()
    probability = contrast_1_count / total_rows
    
    for index, row in df.iterrows():
        print(f"Row {index} {json_col}: {extract_label_from_json(row[json_col])}, {label_col}: {str(row[label_col]) if pd.notna(row[label_col]) else 'NA'}, contrast: {row[new_col_name]}")
    
    print(f"Total rows: {total_rows}")
    print(f"Number of rows where contrast is 1: {contrast_1_count}")
    print(f"Probability of contrast being 1: {probability:.2%}")
    print()
    
    return df

# 批量处理文件
for item in os.listdir("未评估数据"):
    input_file_path = os.path.join("未评估数据", item)
    
    # 将文件名按照下划线分割，并取需要的部分
    base_name = '_'.join(item.split('_')[:2])
    
    # 创建新的文件夹路径
    folder_path = os.path.join("已评估数据")
    os.makedirs(folder_path, exist_ok=True)
    
    output_file_path = os.path.join(folder_path, base_name + "_evaluated.xlsx")
    # print(input_file_path, output_file_path)
    if os.path.isfile(input_file_path) and input_file_path.endswith('.xlsx'):
        df = pd.read_excel(input_file_path)
          # 处理数据
        try:
            df["label"] = df["label"].apply(lambda x: str(x) if pd.notna(x) else "NA")
            df = process_columns(df, json_col='predict', label_col='label')
        except KeyError as e:
            print(f"Error processing file {input_file_path}: {e}")
            

        # 保存处理后的数据
        df.to_excel(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")
    else:
        print(f"Skipped non-xlsx file: {input_file_path}")
 

Row 0 predict: 独立练习, label: 独立练习, contrast: 1
Row 1 predict: 独立练习, label: 独立练习, contrast: 1
Row 2 predict: NA, label: NA, contrast: 1
Row 3 predict: 个体发言, label: 个体发言, contrast: 1
Row 4 predict: 个体发言, label: 个体发言, contrast: 1
Row 5 predict: NA, label: NA, contrast: 1
Row 6 predict: 个体发言, label: 个体发言, contrast: 1
Row 7 predict: 独立练习, label: NA, contrast: 0
Row 8 predict: NA, label: NA, contrast: 1
Row 9 predict: NA, label: NA, contrast: 1
Row 10 predict: 个体发言, label: 个体发言, contrast: 1
Row 11 predict: NA, label: NA, contrast: 1
Row 12 predict: NA, label: NA, contrast: 1
Row 13 predict: 小组学习, label: 小组学习, contrast: 1
Row 14 predict: 小组学习, label: 多人展示, contrast: 0
Row 15 predict: NA, label: NA, contrast: 1
Row 16 predict: NA, label: NA, contrast: 1
Row 17 predict: NA, label: NA, contrast: 1
Row 18 predict: NA, label: NA, contrast: 1
Row 19 predict: 个体发言, label: 个体发言, contrast: 1
Row 20 predict: 独立练习, label: 个体发言, contrast: 0
Row 21 predict: NA, label: NA, contrast: 1
Row 22 predict: NA, la

In [ ]:
# 定义一个通用函数来处理提取、比较和统计
def process_columns(df, json_col='predict', label_col='label'):
    new_col_name = f"{json_col}_contrast"

    def process_row(row):
        json_label = extract_label_from_json(row[json_col])
        numeric_label = row[label_col] if pd.notna(row[label_col]) else "NA"
        return compare_labels(json_label, numeric_label)
    if label_col not in df.columns:
        raise KeyError(f"Column '{label_col}' not found in DataFrame.")
    
    df[new_col_name] = df.apply(process_row, axis=1)
    total_rows = len(df)
    contrast_1_count = df[new_col_name].apply(lambda x: x if isinstance(x, int) else 0).sum()
    probability = contrast_1_count / total_rows
    
    for index, row in df.iterrows():
        print(f"Row {index} {json_col}: {extract_label_from_json(row[json_col])}, {label_col}: {str(row[label_col]) if pd.notna(row[label_col]) else 'NA'}, contrast: {row[new_col_name]}")
    
    print(f"Total rows: {total_rows}")
    print(f"Number of rows where contrast is 1: {contrast_1_count}")
    print(f"Probability of contrast being 1: {probability:.2%}")
    print()

In [37]:
df = pd.read_excel('未评估数据/doubao_705_processed_output.xlsx')
type(df.loc[2,"label"])

float

In [13]:
import os

# 批量处理文件
for item in os.listdir("未评估数据"):
    input_file_path = os.path.join("未评估数据", item)
    
    # 将文件名按照下划线分割，并取需要的部分
    base_name = '_'.join(item.split('_')[:2])
    
    # 创建新的文件夹路径
    folder_path = os.path.join("已评估数据")
    os.makedirs(folder_path, exist_ok=True)
    
    output_file_path = os.path.join(folder_path, base_name + "_evaluated.xlsx")
    print(input_file_path, output_file_path)
    

未评估数据\doubao_705_processed_output.xlsx 已评估数据\doubao_705_evaluated.xlsx
未评估数据\doubao_726_processed_output.xlsx 已评估数据\doubao_726_evaluated.xlsx
未评估数据\doubao_labazhou_processed_output.xlsx 已评估数据\doubao_labazhou_evaluated.xlsx
未评估数据\doubao_renzhi_processed_output.xlsx 已评估数据\doubao_renzhi_evaluated.xlsx
未评估数据\doubao_tianchuang2_processed_output.xlsx 已评估数据\doubao_tianchuang2_evaluated.xlsx
未评估数据\doubao_wuhan1_asr_voice_processed_output.xlsx 已评估数据\doubao_wuhan1_evaluated.xlsx
未评估数据\doubao_wuhan2_asr_voice_processed_output.xlsx 已评估数据\doubao_wuhan2_evaluated.xlsx
未评估数据\doubao_yanzi_processed_output.xlsx 已评估数据\doubao_yanzi_evaluated.xlsx
未评估数据\gpt4o_705_processed_output.xlsx 已评估数据\gpt4o_705_evaluated.xlsx
未评估数据\gpt4o_726_processed_output.xlsx 已评估数据\gpt4o_726_evaluated.xlsx
未评估数据\gpt4o_labazhou_processed_output.xlsx 已评估数据\gpt4o_labazhou_evaluated.xlsx
未评估数据\gpt4o_renzhi_processed_output.xlsx 已评估数据\gpt4o_renzhi_evaluated.xlsx
未评估数据\gpt4o_tianchuang2_processed_output.xlsx 已评估数据\gpt4o_tianchuang2_ev

In [60]:
import re
import json
import pandas as pd
df = pd.read_excel('未评估数据/gpt4o_dir_processed_output.xlsx')
# 定义一个函数来提取label值
def extract_label_from_json(content):
    match = re.search(r'\{.*?\}', content, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            json_dict = json.loads(json_str)
            return json_dict.get("label", "Key not found")
        except json.JSONDecodeError:
            return "JSON decode error"
    return "No JSON content"

# 定义一个函数来比较两个label值
def compare_labels(label1, label2):
    labels1 = label1.strip().replace(' ', '').split('、')
    labels2 = label2.strip().replace(' ', '').split('、')
    
    if len(labels1) == 1 and len(labels2) == 1:
        return 1 if labels1[0] == labels2[0] else 0
    else:
        set1 = set(labels1)
        set2 = set(labels2)
        return 1 if set1 & set2 else 0

# 定义一个通用函数来处理提取、比较和统计
def process_columns(df, json_col='predict', label_col='label'):
    new_col_name = f"{json_col}_contrast"

    def process_row(row):
        json_label = extract_label_from_json(row[json_col])
        numeric_label = str(row[label_col]) if pd.notna(row[label_col]) else "NA"
        return compare_labels(json_label, numeric_label)
    
    df[new_col_name] = df.apply(process_row, axis=1)
    total_rows = len(df)
    contrast_1_count = df[new_col_name].sum()
    probability = contrast_1_count / total_rows
    
    print(f"Comparing {json_col} and {label_col}:")
    for index, row in df.iterrows():
        print(f"Row {index} {json_col}: {extract_label_from_json(row[json_col])}, {label_col}: {str(row[label_col]) if pd.notna(row[label_col]) else 'NA'}, contrast: {row[new_col_name]}")
    
    print(f"Total rows: {total_rows}")
    print(f"Number of rows where contrast is 1: {contrast_1_count}")
    print(f"Probability of contrast being 1: {probability:.2%}")
    print()

# 示例调用
# df = pd.DataFrame(...) # 创建你的DataFrame
# process_columns(df, 'gpt4o_output_new')


In [61]:
# 使用通用函数处理两组列
process_columns(df)

Comparing predict and label:
Row 0 predict: NA, label: NA, contrast: 1
Row 1 predict: 独立练习, label: 独立练习, contrast: 1
Row 2 predict: 独立练习, label: 独立练习, contrast: 1
Row 3 predict: 个体发言, label: NA, contrast: 0
Row 4 predict: 个体发言, label: NA, contrast: 0
Row 5 predict: NA, label: 独立练习, contrast: 0
Row 6 predict: 独立练习, label: 独立练习, contrast: 1
Row 7 predict: 个体发言, label: 个体发言, contrast: 1
Row 8 predict: NA, label: NA, contrast: 1
Row 9 predict: NA, label: NA, contrast: 1
Row 10 predict: 独立练习, label: 独立练习, contrast: 1
Row 11 predict: 个体发言, label: NA, contrast: 0
Row 12 predict: 个体发言, label: 个体发言, contrast: 1
Row 13 predict: NA, label: NA, contrast: 1
Row 14 predict: NA, label: NA, contrast: 1
Row 15 predict: 个体展示, label: 个体展示, contrast: 1
Row 16 predict: 个体发言, label: NA, contrast: 0
Row 17 predict: NA, label: NA, contrast: 1
Row 18 predict: 个体发言, label: 个体发言, contrast: 1
Row 19 predict: NA, label: NA, contrast: 1
Row 20 predict: NA, label: NA, contrast: 1
Row 21 predict: 独立练习, label: NA, con

In [62]:
df

,S_T,E_T,text,label,predict,predict_contrast
0,0分45秒,0分57秒,好，准备好了吗？你们好，上课，可以哈，好，请坐，那大家看到这个题目，,NaN,分析过程：在这段话语中，“好，准备好了吗？”是老师在问学生是否已经准备好上课。“你们好，上课...,1
1,1分4秒,1分9秒,那同学们能不能利用这八个字组成两个成语，,独立练习,分析过程：在这段话语中，“那同学们能不能利用这八个字组成两个成语”体现出老师发起了一个开展课...,1
2,2分18秒,2分39秒,也就是刚才同学们所讲的一种量，随着另一种量的变化而变化，大家来看一下，倒果汁是不是我们常见的...,独立练习,分析过程：在这段话语中，“你想一想”体现出老师发起了一个开展课堂活动的指令。“这里面会存在哪...,1
3,3分20秒,3分41秒,好吃嗯，果果汁是就果汁的水水水位线水位线，那我们就是什么？高高度，那高度和谁呢？还有一个变量...,NaN,分析过程：在这段话语中，“那谁也会发生一些变化呢？”、“果汁的高度在增加，那谁也会发生一些变...,0
4,4分36秒,4分55秒,那随着果汁高度的增高，我们在六年级上学期所学的什么也在变，那个大大大果果汁的体积体积也在变好...,NaN,分析过程：在这段话语中，“谁是随着谁的变化而变化的呢”体现出老师发起了一个开展课堂活动的指令...,0
5,5分9秒,5分32秒,就是非果汁体积会随着非洲果汁的高度的变化而变化。好，带着这个知识点，我们来看一道例题，一辆汽...,独立练习,分析过程：在这段话语中，“好，带着这个知识点，我们来看一道例题”体现出老师将学生的注意力转向...,0
6,5分35秒,5分44秒,咋说，时间和路路程，时间和路程。观察上表回答一下下面的问题，,独立练习,分析过程：在这段话语中，“观察上表回答一下下面的问题”体现出老师发起了一个开展课堂活动的指令...,1
7,5分47秒,5分49秒,谁来说一说，,个体发言,分析过程：在这段话语中，“谁来说一说”体现出老师发起了一个开展课堂活动的指令，邀请某个学生来...,1
8,5分56秒,6分7秒,刚才的第一个问题已经其实已经被我们班同学已经发现过了，是哪两个量，时间和路程在表中都呈现给大家了。,NaN,分析过程：在这段话语中，”刚才的第一个问题其实已经被我们班同学发现过了，是哪两个量“体现出老...,1
9,6分11秒,6分13秒,随着时间变化的呢，,NaN,分析过程：在这段话语中，“随着时间变化的呢”体现出老师在陈述、引导或提问，然而这句话没有明显...,1


In [63]:
df['label'] =df['label'].apply(lambda x: str(x) if pd.notna(x) else "NA")
df.to_excel('已评估数据/gpt4o_dir_evaluated.xlsx', index=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\zonekey008\Desktop\class_acitvity_identify\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zonekey008\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

{'input_ids': [[101, 7592, 1010, 2129, 2024, 2017, 1029, 102, 100, 100], [101, 1045, 1005, 1049, 2986, 1010, 4067, 2017, 999, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
